In [1]:
import torch


In [ ]:
class SkipGramFoo(torch.nn.Module):
  def __init__(self, voc: int, emb: int, _):
    super().__init__()
    self.emb = torch.nn.Embedding(num_embeddings=voc, embedding_dim=emb)
    self.ffw = torch.nn.Linear(in_features=emb, out_features=voc, bias=False)
    self.sig = torch.nn.Sigmoid()

  def forward(self, inpt, trgs, rand):
    emb = self.emb(inpt)
    ctx = self.ffw.weight[trgs]
    rnd = self.ffw.weight[rand]
    out = torch.bmm(ctx, emb.unsqueeze(-1)).squeeze()
    rnd = torch.bmm(rnd, emb.unsqueeze(-1)).squeeze()
    out = self.sig(out)
    rnd = self.sig(rnd)
    pst = -out.log().mean()
    ngt = -(1 - rnd + 10**(-3)).log().mean()
    return pst + ngt

In [ ]:
args = (len(words_to_ids), 64, 2)
mFoo = SkipGramFoo(*args)
print('mFoo', sum(p.numel() for p in mFoo.parameters()))
opFoo = torch.optim.Adam(mFoo.parameters(), lr=0.003)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
%pip install torchinfo
from torchinfo import summary

mFoo

In [ ]:
windows = list(more_itertools.windowed(tokens, 3))
inputs = [w[1] for w in windows]
targets = [[w[0], w[2]] for w in windows]
input_tensor = torch.LongTensor(inputs)
target_tensor = torch.LongTensor(targets)
dataset = torch.utils.data.TensorDataset(input_tensor, target_tensor)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=True)

In [ ]:
wandb.init(project='mlx6-word2vec', name='mFoo')
mFoo.to(device)
for epoch in range(2):
  prgs = tqdm.tqdm(dataloader, desc=f"Epoch {epoch+1}", leave=False)
  for inpt, trgs in prgs:
    inpt, trgs = inpt.to(device), trgs.to(device)
    rand = torch.randint(0, len(words_to_ids), (inpt.size(0), 2)).to(device)
    opFoo.zero_grad()
    loss = mFoo(inpt, trgs, rand)
    loss.backward()
    opFoo.step()
    wandb.log({'loss': loss.item()})

In [ ]:
import torch
import torch.nn as nn

class PositiveContextTower(nn.Module):
    def __init__(self, vocab_size, embed_dim, lstm_hidden_dim, output_dim):
        """
        :param vocab_size: Size of the vocabulary.
        :param embed_dim: Dimensionality of token embeddings.
        :param lstm_hidden_dim: Number of hidden units in the LSTM.
        :param output_dim: Final embedding dimension (common space).
        """
        super(PositiveContextTower, self).__init__()
        # Maps token indices to embeddings (each of size embed_dim)
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        # LSTM that processes sequences of embeddings.
        # With batch_first=True, input shape: (batch_size, sequence_length, embed_dim)
        self.lstm = nn.LSTM(embed_dim, lstm_hidden_dim, batch_first=True)
        # Final projection to the common embedding space
        self.fc = nn.Linear(lstm_hidden_dim, output_dim)
    
    def forward(self, x):
        """
        :param x: Tensor of shape (batch_size, sequence_length) containing token indices.
        :return: Tensor of shape (batch_size, output_dim) representing the sentence embedding.
        """
        # Convert token indices to embeddings.
        # Output shape: (batch_size, sequence_length, embed_dim)
        x = self.embedding(x)
        
        # The LSTM processes the sequence in order (along the sequence_length dimension)
        # 'output' has shape (batch_size, sequence_length, lstm_hidden_dim)
        # 'hn' (hidden state) has shape (num_layers, batch_size, lstm_hidden_dim)
        output, (hn, cn) = self.lstm(x)
        
        # Use the final hidden state from the last LSTM layer.
        # This represents the entire sentence after processing all tokens.
        last_hidden = hn[-1]  # Shape: (batch_size, lstm_hidden_dim)
        
        # Project the LSTM output into the common embedding space.
        return self.fc(last_hidden)


In [ ]:
class QueryTower(nn.Module):
    def __init__(self, vocab_size, embed_dim, lstm_hidden_dim, output_dim):
        """
        :param vocab_size: Size of the vocabulary.
        :param embed_dim: Dimensionality of token embeddings.
        :param lstm_hidden_dim: Number of hidden units in the LSTM.
        :param output_dim: Final embedding dimension (common space).
        """
        super(QueryTower, self).__init__()
        # Maps token indices to embeddings (each of size embed_dim)
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        # LSTM that processes sequences of embeddings.
        # With batch_first=True, input shape: (batch_size, sequence_length, embed_dim)
        self.lstm = nn.LSTM(embed_dim, lstm_hidden_dim, batch_first=True)
        # Final projection to the common embedding space
        self.fc = nn.Linear(lstm_hidden_dim, output_dim)
    
    def forward(self, x):
        """
        :param x: Tensor of shape (batch_size, sequence_length) containing token indices.
        :return: Tensor of shape (batch_size, output_dim) representing the sentence embedding.
        """
        # Convert token indices to embeddings.
        # Output shape: (batch_size, sequence_length, embed_dim)
        x = self.embedding(x)
        
        # The LSTM processes the sequence in order (along the sequence_length dimension)
        # 'output' has shape (batch_size, sequence_length, lstm_hidden_dim)
        # 'hn' (hidden state) has shape (num_layers, batch_size, lstm_hidden_dim)
        output, (hn, cn) = self.lstm(x)
        
        # Use the final hidden state from the last LSTM layer.
        # This represents the entire sentence after processing all tokens.
        last_hidden = hn[-1]  # Shape: (batch_size, lstm_hidden_dim)
        
        # Project the LSTM output into the common embedding space.
        return self.fc(last_hidden)


In [6]:
#prepare the data for the query tower work on the query triplets
import json
with open('/Users/yuliagoryachev/Documents/mlx/mlx_week2/two-tower-search/data/train_triples_sample.json') as f:
    data = json.load(f)

In [9]:
data

[{'query': ')what was the immediate impact of the success of the manhattan project?',
  'positive': 'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.',
  'positive_url': 'http://www.pitt.edu/~sdb14/atombomb.html',
  'negative': 'Parents and caregivers, can make the personal choice to reduce exposures of their infants and children to BPA: 1  Don’t microwave polycarbonate plastic food containers. 2  Plastic containers have recycle codes on the bottom. 3  Reduce your use of canned foods.',
  'negative_url': 'https://www.niehs.nih.gov/health/topics/agents/sya-bpa/index.cfm'},
 {'query': ')what was the immediate impact of the success of the manhattan project?',
  'positive': 'The Manhattan Project and its atomic bomb h

In [10]:
average_sentence_length = sum(len(item['query'].split()) for item in data) / len(data)
print(average_sentence_length)#this before stemming and removing stopwords
average_context_length = sum(len(item['positive'].split()) for item in data) / len(data)
print(average_context_length)#this before stemming and removing stopwords

12.0
42.8


In [12]:
from tqdm import tqdm
from utils import tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yuliagoryachev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
#data preparartion

In [33]:
SENT_LEN = 12
CONTEXT_LEN = 50

def pad_truncate(data, my_key, max_len):
    res = []
    for d in tqdm(data):
        clean_sent = tokenize(d[my_key], {})
        tokens = clean_sent[:max_len]
        tokens += ['<unk>'] * (max_len - len(tokens))
        res.append(tokens)
    return res


In [34]:
padded_query = pad_truncate(data, 'query', SENT_LEN)
padded_positive = pad_truncate(data, 'positive', CONTEXT_LEN)
padded_negative = pad_truncate(data, 'negative', CONTEXT_LEN)

100%|██████████| 5/5 [00:00<00:00, 10205.12it/s]


In [25]:
padded_query

[['immediate',
  'impact',
  'success',
  'manhattan',
  'project',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>'],
 ['immediate',
  'impact',
  'success',
  'manhattan',
  'project',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>'],
 ['immediate',
  'impact',
  'success',
  'manhattan',
  'project',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>'],
 ['immediate',
  'impact',
  'success',
  'manhattan',
  'project',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>'],
 ['immediate',
  'impact',
  'success',
  'manhattan',
  'project',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>']]

In [24]:
#convert to indices using the word_to_id dictionary
import joblib
word_to_id = joblib.load('/Users/yuliagoryachev/Documents/mlx/mlx_week2/two-tower-search/eda/word_to_ids.pkl')

In [36]:
#convert to indices
def convert_to_indices(data, word_to_id):
    res = []
    for d in tqdm(data):
        res.append([word_to_id.get(w, word_to_id['<unk>']) for w in d])
    return res

In [39]:
padded_query = convert_to_indices(padded_query, word_to_id)
padded_positive = convert_to_indices(padded_positive, word_to_id)
padded_negative = convert_to_indices(padded_negative, word_to_id)

100%|██████████| 5/5 [00:00<00:00, 2954.98it/s]


In [38]:
padded_query

[[305701, 636225, 277834, 180836, 360087, 0, 0, 0, 0, 0, 0, 0],
 [305701, 636225, 277834, 180836, 360087, 0, 0, 0, 0, 0, 0, 0],
 [305701, 636225, 277834, 180836, 360087, 0, 0, 0, 0, 0, 0, 0],
 [305701, 636225, 277834, 180836, 360087, 0, 0, 0, 0, 0, 0, 0],
 [305701, 636225, 277834, 180836, 360087, 0, 0, 0, 0, 0, 0, 0]]

In [40]:
padded_positive

[[817869,
  627498,
  1033720,
  619830,
  880959,
  775439,
  1042134,
  277834,
  180836,
  360087,
  619830,
  1078582,
  579722,
  563833,
  1011756,
  95939,
  737175,
  435927,
  760362,
  277834,
  667363,
  187746,
  531349,
  434897,
  537731,
  526049,
  1035483,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [180836,
  360087,
  737175,
  923907,
  364884,
  26479,
  1144446,
  512957,
  249430,
  866075,
  300748,
  656421,
  237767,
  737175,
  453360,
  19465,
  636225,
  349661,
  882790,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1145775,
  180836,
  360087,
  180836,
  360087,
  180836,
  360087,
  673889,
  457358,
  737175,
  923907,
  377818,
  277834,
  360087,
  568307,
  530354,
  28326,
  512957,
  530354,
  457358,
  896883,
  971496,
  343410,
  19108,
  0,
  0,
  0,
  0,
  0,
 